Use Existing SQL Warehouse (5 mins)

Go to SQL → SQL Editor

Top-right → Warehouse dropdown

Select:


![](/Volumes/workspace/ecommerce/ecommerce_data/SQL Warehouse.png)
![](/Volumes/workspace/ecommerce/ecommerce_data/SQL Warehouse Starter.png)

#Query 1: Daily Revenue with Rolling 7-Day Average
##CTE + window function + time-series analytics


In [0]:
%sql
WITH daily_revenue AS (
    SELECT
        CAST(event_time AS DATE) AS event_date,
        SUM(price) AS revenue
    FROM silver.events
    WHERE event_type = 'purchase'
      AND price IS NOT NULL
    GROUP BY CAST(event_time AS DATE)
)
SELECT
    event_date,
    revenue,
    ROUND(
        AVG(revenue) OVER (
            ORDER BY event_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ), 2
    ) AS revenue_7day_moving_avg
FROM daily_revenue
ORDER BY event_date;


#Query 2: Category-Level Conversion Funnel
##Funnel analysis + conditional aggregation


In [0]:
%sql
WITH category_events AS (
    SELECT
        category_code,
        SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS views,
        SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS purchases
    FROM silver.events
    GROUP BY category_code
)
SELECT
    category_code,
    views,
    purchases,
    ROUND(
        purchases * 100.0 / NULLIF(views, 0),
        2
    ) AS conversion_rate_percent
FROM category_events
ORDER BY conversion_rate_percent DESC;


In [0]:
%sql


In [0]:
%sql
SELECT
    product_id,
    category_code,
    SUM(price) AS total_revenue
FROM silver.events
WHERE event_type = 'purchase'
GROUP BY product_id, category_code
ORDER BY total_revenue DESC
LIMIT 10;


#Query 4: Customer Purchase Segmentation (LTV Logic)
##Nested aggregation + CASE logic


In [0]:
%sql
WITH customer_metrics AS (
    SELECT
        user_id,
        COUNT(*) AS purchase_count,
        SUM(price) AS lifetime_value
    FROM silver.events
    WHERE event_type = 'purchase'
    GROUP BY user_id
)
SELECT
    CASE
        WHEN purchase_count >= 10 THEN 'VIP'
        WHEN purchase_count >= 5 THEN 'Loyal'
        ELSE 'Regular'
    END AS customer_segment,
    COUNT(*) AS customer_count,
    ROUND(AVG(lifetime_value), 2) AS avg_ltv
FROM customer_metrics
GROUP BY customer_segment;


#Query 5: Daily Active Customers (Advanced KPI)
##Distinct count + temporal grouping


In [0]:
%sql
SELECT
    CAST(event_time AS DATE) AS event_date,
    COUNT(DISTINCT user_id) AS daily_active_users
FROM silver.events
GROUP BY CAST(event_time AS DATE)
ORDER BY event_date;

